# Assignment 2. Moskalev Artem.

## Problem 1

$minimize(4x_1 + 5|x_2-1|)$, subject to $|2x_1|+|x_2-3|<=5$
<br>
<br>
Let's substitute modules:
<br>
<br>
$|x_2-1|=z_1$, $z_1>=x_2-1$, $z_1>=-x_2+1$
<br>
<br>
$|2x_1|=z_2$, $z_2>=-2x_1$, $z_2>=2x_1$
<br>
<br>
$|x_2-3|=z_3$, $z_3>=x_2-3$, $z_3>=-x_2+3$
<br><br>
Thus, we come to the following form:<br>
<br>
$minimize(4x_1+5z_1)$, s.t.<br>
$z_1>=x_2-1$<br>
$z_1>=-x_2+1$<br>
$z_2>=2x_1$<br>
$z_2>=-2x_1$<br>
$z_3>=x_2-3$<br>
$z_3>=-x_2+3$<br>
$z_1+z_2<=5$<br>
<br>
Next, we can reformulate constraints to come to more suitable form: $minimize(c^Tx)$, s.t. $Ax>=b$:<br>

$z_1-x_2>=-1$<br>
$z_1+x_2>=1$<br>
$z_2-2x_1>=0$<br>
$z_2+2x_1>=0$<br>
$z_3-x_2>=-3$<br>
$z_3+x_2>=3$<br>
$-z_1+-z_2>=-5$<br>
<br>
So, we have decision variables $x_1,x_2,z_1,z_2,z_3$, cost vector $c=(4,0,5,0,0)^T$, matrix A which holds left hand side of constraint inequalities and vector $b=(-1,1,0,0,-3,3,-5)^T$ for right hand side. Now, we can use cvxpy to perform an optimization:

In [4]:
from cvxpy import *
import numpy as np

x=Variable(5) #here are our x1,x2,z1,z2,z3
c = np.asarray([4., 0., 5., 0., 0.]); #cost vector

A = np.asarray([[ 0., -1.,  1.,  0., 0.],
                [ 0.,  1.,  1.,  0., 0.],
                [-2.,  0.,  0.,  1., 0.],
                [ 2.,  0.,  0.,  1., 0.],
                [ 0., -1.,  0.,  0., 1.], 
                [ 0.,  1.,  0.,  0., 1.], 
                [ 0.,  0., 0., -1., -1.]]);

b = np.asarray([-1., 1., 0., 0.,-3.,3.,-5.]);

constraints = [A*x>=b]
obj = Minimize(c*x)

prob = Problem(obj, constraints)
prob.solve()

print("status:", prob.status)
print("optimal value", prob.value)
print("optimal x_1 =", round(float(x.value[0]),2),'optimal x_2 =',int(x.value[1]))

status: optimal
optimal value -5.99999999851067
optimal x_1 = -1.5 optimal x_2 = 1


## Problem 2

There are many ways to formulate this problem. One of them is to minimize an amount of power with constraint that all desired demands are satisfied. Also, we need to restrict that all our powers are greater or equal than zero.

$\min\sum_{i=1}^mp_i$, s.t. <br><br>
$\sum_{j=1}^ma_{i,j}p_j>=I_j^*$ for     $1<=i<=n$ <br> $p_j>=0$ for $1<=j<=m$

In [7]:
np.random.seed(7766)

n = 7 #segments
m = 5 #lamps

a_nod = np.random.random((n, m))
#rounder = np.vectorize(lambda x: int(x))
#a_nod=rounder(a_nod) 

I_desired = np.random.random((n, 1))
#I_desired = rounder(I_desired)

p_i = Variable(m)
constraints = []

for each_segment in range(n): #here we add constraints that the demand of each segment is satisfied
    constraints.append(sum_entries(a_nod[each_segment]@p_i)>=I_desired[each_segment])
    
constraints.append(p_i>=0)
obj = Minimize(sum_entries(p_i)) #so, we want the total power to be as small as possibe
prob = Problem(obj, constraints)
prob.solve()

print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", p_i.value)

status: optimal
optimal value 1.2834433268332919
optimal var [[ -2.63142284e-11]
 [  1.84360592e-01]
 [  5.98942883e-01]
 [  1.06225317e-11]
 [  5.00139852e-01]]


P.S. Variable values with powers like $e^{-11}$ should be treated like zeros here

Second way to formulate this problem is to minimize the difference between desired demand and actual demand with constraint that all segments are sufficienlty lighted $(I_i>=I_i^*)$. However, in this case, we will need to introduce at least $n$ new variables and $2n$ new constraints to substitute the module, so it won't be the most efficient way.

- $\min\sum_{i=1}^n|I^*_i-I_i|$, s.t. $p_j>=0$ and $I_i>=I_i^*$

The third way is to work with $\|.\|_ {\infty}$ and to minimize the maximum difference between desired and actual demand: <br>
<br>
- $\min(\max|I_i^*-I_i|)$ s.t. $I_i>= I_i^*$ for $1<=i<=n$ and $p_j>=0$ for $1<=j<=m$
<br>

<br>
However, here we also face a need to introduce new variables and constraints. So, we remain our first formulation $\min\sum_{i=1}^mp_i$ as the main one.

## Problem 3

#### Part 1

Let $P$ be the profit, hence in our case we have $P=n(9-1.2)+m(8-0.9)=7.8n+7.1m$, where $n$ is number of products of the first type and $m$ is number of products of the second type and obviously: $m,n>=0$.<br>
<br>
We also should introduce additional constraints related to the time of production: time for assembling and for testing of both products can not be higher than than a particular limit: $\frac14n+\frac13m<=90$ - time for assembling, $\frac18n+\frac13m<=80$ time for testing. Thus, we have:<br>
<br>
$$\max(7.8n+7.1m)$$
<center>, s.t. <br>
<br>
$$\frac14n+\frac13m<=90$$ <br>
$$\frac18n+\frac13m<=80$$ <br>
$$m,n>=0$$

In some cases our decision variables might be treated as integers (If company produces briks or cars). However, since we have no idea of what company's products are, let's assume that they company can produce fractional number of product (oil, flour etc.)

In [9]:
n = Variable()
m = Variable()


constraints=[m>=0,n>=0,(n/4)+(m/3)<=90, (n/8)+(m/3)<=80]
obj = Maximize(7.8*n+7.1*m) #so, we want the total power to be as small as possibe
prob = Problem(obj, constraints)
prob.solve()

print("status:", prob.status)
print("optimal value", prob.value)
print("optimal n:", n.value)
print("optimal m:", m.value)

status: optimal
optimal value 2807.9999882490756
optimal n: 359.999995524
optimal m: 3.26239653845e-06


#### Part 2(a)

Let's now introduce extra time $e$ for assembling and each hour of extra work will reduce company's profit for 7 dollars, hence we have:<br>
<br>
$$\max(7.8n+7.1m-7e)$$
<center>, s.t. <br>
<br>
$$\frac14n+\frac13m<=90+e$$ <br>
$$\frac18n+\frac13m<=80$$ <br>
$$m,n>=0$$
$$e>=0$$
$$e<=50$$

In [10]:
n = Variable()
m = Variable()
e = Variable()


constraints=[m>=0,n>=0,(n/4)+(m/3)<=90, (n/8)+(m/3)<=80, e>=0, e<=50]
obj = Maximize(7.8*n+7.1*m-7*e) #so, we want the total power to be as small as possibe
prob = Problem(obj, constraints)
prob.solve()

print("status:", prob.status)
print("optimal value", prob.value)
print("optimal n:", round(n.value))
print("optimal m:", round(m.value))
print("optimal e:", e.value)

status: optimal
optimal value 2807.9999995638636
optimal n: 360.0
optimal m: 0.0
optimal e: 3.08613292347e-08


From here, we can deduce that introducing extra time does not change situation for much.

#### Part 2(b)

Let's now think how our problem will look like if we introdce a discount over the bill of raw materials if it is more than 300$

$P = \begin{cases} 9n+8m-c, & \mbox{if } n<300 \\ 9n+8m-0.9c, & \mbox{if } n>=300 \end{cases}$
<br>
where $c = (1.2n+0.9m)$ - cost of production
<br>, s.t.<br>
<br>
$\frac14n+\frac13m<=90$ <br>
$\frac18n+\frac13m<=80$ <br>
$m,n>=0$

I feel that it won't be correct to formulate this whole problem as LPP, because either company buys raw materials for more than 300$ or less. Instead, I suggest solving to distict LP problems: with and without the discont

##### With discount:

$$\max(9n+8m-0.9(1.2n+0.9m))$$
<center>, s.t. <br>
<br>
$$\frac14n+\frac13m<=90$$ <br>
$$\frac18n+\frac13m<=80$$ <br>
$$m,n>=0$$

In [12]:
n = Variable()
m = Variable()


constraints=[(n/4)+(m/3)<=90, (n/8)+(m/3)<=80,m>=0,n>=0]
obj = Maximize(9*n+8*m-0.9*(1.2*n+0.9*m)) #so, we want the total power to be as small as possibe
prob = Problem(obj, constraints)
prob.solve()

print("status:", prob.status)
print("optimal value", prob.value)
print("optimal n:", n.value)
print("optimal m:", m.value)

status: optimal
optimal value 2851.199988900705
optimal n: 359.999995874
optimal m: 3.0009176932e-06


##### Without discount:

Without discount this problem just turns into that one, we solved in Part 1.

## Problem 4

<img src="problem_4_1.jpg">
<img src="problem_4_2.jpg">